In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
sns.set(style="white", context='poster')
from sympy import *
import math
init_printing(use_unicode=True)
from scipy.integrate import solve_ivp

In [2]:
%load_ext autoreload
%autoreload 2


In [43]:
#from model_equations_separate_NC import *
from model_equations_separate_NC_sep_vmax import *


In [44]:
import run_sensitivity

In [30]:
    param_df = pd.read_excel( 'Model_Parameters.xlsx',)

In [31]:
param_df.columns

Index(['parameter', 'units', 'description', 'organism', 'full model',
       'full model per day', 'MIN model hardcoded parameters',
       'LEAK model hardcoded parameters', 'MIXO model hardcoded parameters',
       'Tunable parameters (PRO fitting)', 'Tunable parameters (HET fitting)',
       'lower bound', 'upper bound', 'logscale fitting'],
      dtype='object')

In [32]:
model_name = 'MIN'
organism = 'PRO'
""" return 
params_to_update (list of strings)
bounds (list of pairs)
log_params (list of bools
"""
param_df = pd.read_excel( 'Model_Parameters.xlsx',)
param_df['values'] = param_df['full model']
if model_name is not None:
    model_col = f'{model_name} model hardcoded parameters'
    param_df.loc[~param_df[model_col].isna(), 'values'] = param_df.loc[~param_df[model_col].isna(), model_col] 
param_vals_series = param_df['values']
param_vals_series.index = param_df.parameter


In [33]:
    model_col = f'{model_name} model hardcoded parameters'
    org_col = f'Tunable parameters ({organism} fitting)'
    model_col
    #param_df.loc[]

'MIN model hardcoded parameters'

In [34]:
tunable_param_df = param_df.loc[param_df[model_col].isna() & 
             param_df[org_col].isin(['Yes'])] 


In [35]:
list(zip(tunable_param_df['lower bound'],tunable_param_df['upper bound']))

In [40]:
def get_param_tuning_values(model_name, organism_to_tune):
    """ return 
    params_to_update (list of strings)
    bounds (list of pairs)
    log_params (list of bools
    """
    param_df = pd.read_excel( 'Model_Parameters.xlsx',)
    org_col = f'Tunable parameters ({organism_to_tune} fitting)'

    if model_name not in [None, 'FULL']:
        # restrict to given model
        model_col = f'{model_name} model hardcoded parameters'
        tunable_param_df = param_df.loc[param_df[model_col].isna() & param_df[org_col].isin(['Yes'])] 
    else: 
        # take the full model
        tunable_param_df = param_df.loc[param_df[org_col].isin(['Yes'])] 

    params_to_update = list(tunable_param_df['parameter'])    
    bounds = list(zip(tunable_param_df['lower bound'],tunable_param_df['upper bound']))    
    
    log_params = list(tunable_param_df['logscale fitting'].map({'Yes': True, 'No': 'False'}))                

    return (params_to_update, bounds, log_params)


In [42]:
get_param_tuning_values('FULL', 'HET')

(['gammaDh',
  'EOh',
  'KONh',
  'KINh',
  'KOCh',
  'VmaxONh',
  'VmaxINh',
  'VmaxOCh',
  'E_ROSh',
  'VmaxROSh',
  'K_ROSh',
  'omegaH',
  'KABh',
  'EABh',
  'decayABh',
  'MABh'],
 [(0.0, 1.0),
  (0.0, 1.157407407407407e-05),
  (0.02502723352076287, 2.502723352076287),
  (0.02502723352076287, 2.502723352076287),
  (0.02502723352076287, 2.502723352076287),
  (1.50462962962963e-06, 0.000150462962962963),
  (1.50462962962963e-06, 0.000150462962962963),
  (6.770833333333335e-06, 0.0006770833333333335),
  (4.050925925925926e-08, 4.050925925925926e-06),
  (7.696759259259261e-07, 7.69675925925926e-05),
  (0.02502723352076287, 2.502723352076287),
  (1.0000000000000002e-06, 0.0001),
  (0.01427507995452611, 1.4275079954526109),
  (4.050925925925925e-18, 4.0509259259259247e-16),
  (1.1574074074074074e-08, 1.1574074074074074e-06),
  (1.15740740740741e-08, 1.1574074074074101e-06)],
 ['False',
  'False',
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  